In [1]:
import pandas as pd
import numpy as np
import keras_metrics



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras import utils as k
from tensorflow.keras.callbacks import EarlyStopping



file= 'infarto.csv'
dt=pd.read_csv(file, sep=";")


#Aunque no seamos expertos en el tema entendemos que las columnas casado y trabajo no aportan nada para el modelo
#La residencia si porque la calidad del aire es diferente

dt=dt.drop(['id','casado', 'trabajo'], axis=1)

print("Numero de filas con NaN values:\n",dt.isnull().sum())
print("\n\nComo no hay muchos NaN las filas que cuenten con valores nulos las descartamos")
#dt=dt.drop(dt.index[1000:len(dt)])
dt=dt.dropna()
dt.head(5)

Numero de filas con NaN values:
 genero                    0
edad                      0
hipertensión              0
enfermedad_coronaria      0
residencia                0
nivel_glucosa             0
bmi                     201
uso_tabaco                0
infarto                   0
dtype: int64


Como no hay muchos NaN las filas que cuenten con valores nulos las descartamos


,genero,edad,hipertensión,enfermedad_coronaria,residencia,nivel_glucosa,bmi,uso_tabaco,infarto
0,Masculino,67.0,0,1,Urbana,228.69,36.6,Ex fumador,1
2,Masculino,80.0,0,1,Rural,105.92,32.5,Nunca,1
3,Femenino,49.0,0,0,Urbana,171.23,34.4,Fumador,1
4,Femenino,79.0,1,0,Rural,174.12,24.0,Nunca,1
5,Masculino,81.0,0,0,Urbana,186.21,29.0,Ex fumador,1


In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#Transformamos las columnas necesarias con to_categorical
#antes 
columna1=np.array(dt["genero"])
columna2=np.array(dt["residencia"])
columna3=np.array(dt["uso_tabaco"])
label_encoder = LabelEncoder()

genero = label_encoder.fit_transform(columna1)
residencia= label_encoder.fit_transform(columna2)
tabaco= label_encoder.fit_transform(columna3)


dt["genero"]= k.to_categorical(genero)
dt["residencia"]= k.to_categorical(residencia)
dt["uso_tabaco"]= k.to_categorical(tabaco)

In [3]:
#NORMALIZAMOS LOS DATOS POR COLUMNAS
def minmax_norm(datos_input): 
    return (datos_input - dt.min()) / ( dt.max() - dt.min())
dt=minmax_norm(dt)
dt.head(5)

#OBTENEMOS EL CONJUNTO DE PRUEBA
esperada= dt.iloc[:, 8]
esperada

datos= dt.iloc[:,0:8]
datos

,genero,edad,hipertensión,enfermedad_coronaria,residencia,nivel_glucosa,bmi,uso_tabaco
0,0.0,0.816895,0.0,1.0,0.0,0.801265,0.301260,1.0
2,0.0,0.975586,0.0,1.0,1.0,0.234512,0.254296,0.0
3,1.0,0.597168,0.0,0.0,0.0,0.536008,0.276060,0.0
4,1.0,0.963379,1.0,0.0,1.0,0.549349,0.156930,0.0
5,0.0,0.987793,0.0,0.0,0.0,0.605161,0.214204,1.0
...,...,...,...,...,...,...,...,...
5104,1.0,0.157715,0.0,0.0,1.0,0.221402,0.095074,0.0
5106,1.0,0.987793,0.0,0.0,0.0,0.323516,0.340206,0.0
5107,1.0,0.426270,0.0,0.0,1.0,0.128658,0.232532,0.0
5108,0.0,0.621582,0.0,0.0,1.0,0.513203,0.175258,1.0


In [4]:


#ENTRENAMIENTO, VALIDACION
x_train, x_resto, y_train, y_resto= train_test_split(datos[209:], esperada[209:], test_size=0.2)

x_test, x_pred, y_test, y_pred = train_test_split(x_resto, y_resto, test_size=0.5)



x_in_train, x_in_resto,y_in_train,y_in_resto = train_test_split(datos[:209], esperada[:209], test_size=0.2)
x_in_test, x_in_pred,y_in_test,y_in_pred = train_test_split(x_in_resto, y_in_resto, test_size=0.2)



x_train=pd.concat([x_in_train, x_train], axis=0)
x_test=pd.concat([x_in_test, x_test], axis=0)
x_pred=pd.concat([x_in_pred, x_pred], axis=0)

y_train=pd.concat([y_in_train, y_train], axis=0)
y_test=pd.concat([y_in_test, y_test], axis=0)
y_pred=pd.concat([y_in_pred, y_pred], axis=0)

In [5]:
#PROCEDEMOS A CREAR LA RED

model=Sequential()

#APLICANDO LA FORMULA SABEMOS QUE EN LA CAPA OCULTA DEBEMOS TENER ENTRE 245 Y 981 NEURONAS
model.add(Dense(150, 'sigmoid', 8))
model.add(Dense(300, 'relu'))
model.add(Dense(1, 'sigmoid'))# UNA SALIDA (0/1)

model.compile(loss="mean_squared_error", optimizer='adam',metrics="accuracy")
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
model.fit(x=x_train,y=y_train,validation_split= 0.2, epochs=500, shuffle=True, use_multiprocessing=True)
#steps_per_epoch=50
#, callbacks=es

Epoch 1/500
57/99 [================>.............] - ETA: 0s - loss: 0.0775 - accuracy: 0.9366 

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix
y_pred1 = model.predict(x_pred)
y_pred1 = np.argmax(y_pred1, axis=1)

scores = model.evaluate(x_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
print(precision_score(y_pred, y_pred1 , average="macro"))
print(recall_score(y_pred, y_pred1 , average="macro"))

In [ ]:
cm = confusion_matrix(y_pred, y_pred1)
print(cm)
len(x_train)
